# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [1]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.lesoir.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"r-article--link"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [3]:
# Affichage du nombre d'articles récupérés
len(articles)

0

In [4]:
# Affichage des 10 premières entrées
articles[:10]

[]

## Création d'un dataframe avec les liens et les titres des articles


In [5]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title


In [6]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/lesoir_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [7]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')
chapo = soup.find("r-article--chapo").get_text(separator=' ')
content = soup.find("r-article--section").get_text(separator=' ')

print(title)
print("==================================")
print(chapo)
print("==================================")
print(content)

IndexError: single positional indexer is out-of-bounds

### Nettoyage du texte à l'aide d'expressions régulières

In [19]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


  V ladimir Poutine a dit mercredi que les sanctions internationales visant Moscou pour son offensive en Ukraine « peuvent » avoir des conséquences « négatives » à « moyen terme » sur l’économie nationale, après avoir pourtant vanté ces derniers mois l’adaptation de la Russie face à cette nouvelle conjoncture.                function get_dtkCxSegments() { if ('cxSegments' in localStorage) { /* Récupération de cxSegments + initialisation de variables destinées à en limiter la longueur */ let cx = localStorage.getItem("cxSegments"), valueCx = cx.split(","), valueCxLimited = [], charLength = 0, limit = 2000; /* Limitation de la longueur de la chaine de caractère */ for (var i = 0; i < valueCx.length; i++) { charLength = charLength + valueCx[i].length; if (charLength <= limit) { valueCxLimited.push(valueCx[i]); } } return valueCxLimited; } else { return false; } } window.dtkCxSegments = window.dtkCxSegments || get_dtkCxSegments(); var dtkEmbed = document.currentScript.parentNode; var dtkIf

### Création d'un fichier avec le contenu de l'article


In [20]:
with open("../data/lesoir_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{chapo}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp